In [3]:
import pandas as pd
import numpy as np

import pandas as pd
import spacy
import numpy as np
from gensim.models import LdaMulticore
from gensim.models import CoherenceModel
from gensim.corpora.dictionary import Dictionary
from bs4 import BeautifulSoup


In [5]:
df_articles = pd.read_csv('sävsjö_articles.csv')
df_articles.head()
df_articles = df_articles.dropna()
df_articles.head()

,ID,Title,Text
0,2,Aktiespararna säger ja till Gunvorbudet<span> ...,Aktiespararna rekommenderar sina medlemmar att...
2,7,Man fast i en timme i grop<span> - kuriren.nu<...,En man i 60-årsåldern föll ner i en grop som v...
3,8,Mannen fastnade i en grop  i flera timmar<spa...,En man i 60-årsåldern trillade ner i ett grävt...
4,56,En motorcykel,Här är den längre versionen.
5,66,Stulna braskaminer för 50 000 ska värma,Det börjar bli vinter och kallt. Då är det skö...


In [6]:
for col in df_articles.columns[1:]:
    df_articles[col] = df_articles[col].astype(str)
    df_articles[col] = df_articles[col].apply(lambda x: BeautifulSoup(x).get_text())
df_articles = df_articles.replace(r'[^\w\s]|_', '', regex=True)

c:\Users\oskar\AppData\Local\Programs\Python\Python39\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [7]:
nlp = spacy.load("sv_core_news_lg")
stopwords = nlp.Defaults.stop_words

In [8]:
removal= ['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE', 'NUM', 'SYM']
ent_remove = ['LOC','TME','MSR']
tokens = []
entities = []
i = 0 
for summary in nlp.pipe(df_articles['Text']):
   proj_tok = [token.lemma_.lower() for token in summary if token.pos_ not in removal and not token.is_stop and token.is_alpha and token.lemma_.lower() and token.ent_type_ not in ent_remove]
   ents = list(dict.fromkeys([ent.text.lower() for ent in summary.ents if ent.label_ not in ent_remove]))
   for ent in ents:
      if ent not in proj_tok:
         proj_tok.append(ent)
   tokens.append(proj_tok)
   entities.append(ents)

In [9]:
df_copy = df_articles.copy()
df_copy['tokens'] = tokens
df_copy['entities'] = entities
df_copy.iloc[5]['tokens']

['ny',
 'skolchef',
 'heta',
 'stefan',
 'claesson',
 'jobba',
 'börja',
 'basa',
 'läre',
 'elev',
 'vara',
 'klar',
 'vara',
 'förhandlingsfråga',
 'ny',
 'skolchef',
 'hittillsvarande',
 'arbetsgivare',
 'stefan claesson']

In [11]:
df_copy

,ID,Title,Text,tokens,entities
0,2,Aktiespararna säger ja till Gunvorbudet dise,Aktiespararna rekommenderar sina medlemmar att...,"[aktiespararna, rekommenderar, medlem, anta, b...",[gunvor group]
2,7,Man fast i en timme i grop kurirennu,En man i 60årsåldern föll ner i en grop som va...,"[föll, grop, grävd, egen, tomt, timme, ligga, ...",[]
3,8,Mannen fastnade i en grop i flera timmar exp...,En man i 60årsåldern trillade ner i ett grävt ...,"[trilla, gräva, hål, tomt, grop, stund, hitta,...",[david]
4,56,En motorcykel,Här är den längre versionen,"[vara, version]",[]
5,66,Stulna braskaminer för 50 000 ska värma,Det börjar bli vinter och kallt Då är det skön...,"[börja, vinter, vara, skönt, braskamin, visste...",[]
...,...,...,...,...,...
8600,255432,Bringetofta marknad 2022,ewry,[],[]
8601,255465,BILDEXTRA Det vandrande skådespelet i Komstad ...,I en ljuset av en glödorange sol som sänkte si...,"[ljus, glödorange, sol, sänka, horison, onsdag...","[tiden, komstad, annakarin rostedtvid, bo syré..."
8602,255482,Folkdans vid äldreboenden på fredag,Folkdansarna från Sunne kommer till Hagfors im...,"[folkdansarna, folkdans, ordna, gammal, vecka,...",[]
8603,255580,Företagen en viktig motor i kommunens maskineri,Denna text är en insändare och åsikterna i te...,"[text, vara, insändare, åsikt, text, vara, skr...","[småföretagarnas riksförbund, näringslivs ab, ..."


In [34]:
from gensim.models import TfidfModel
vocab = Dictionary(df_copy['tokens'])
len(vocab.token2id)

df_copy["bow"] = [vocab.doc2idx(i) for i in df_copy['tokens']]
df_bow = df_copy[["ID","bow"]].copy()

df_bow.reset_index(drop=True,inplace=True)

df_bow
    


,ID,bow
0,2,"[0, 9, 8, 2, 3, 5, 4, 7, 10, 0, 1, 6]"
1,7,"[13, 14, 15, 12, 20, 19, 17, 18, 11, 16]"
2,8,"[25, 22, 16, 20, 14, 24, 23, 21]"
3,56,"[26, 27]"
4,66,"[29, 38, 26, 36, 28, 39, 30, 34, 32, 37, 31, 3..."
...,...,...
8598,255432,[]
8599,255465,"[470, 90240, 1371, 9827, 90244, 534, 209, 1650..."
8600,255482,"[90256, 90255, 1448, 81, 128, 487, 946, 1481, ..."
8601,255580,"[3491, 26, 32550, 3493, 3491, 26, 90270, 90258..."


In [35]:

from typing import List

def overlap_procentage(a: set,b:set):
    intersection = [value for value in a if value in b]
    total = len(a) + len(b) - len(intersection)
    return len(intersection)/total



def get_best_overlap(df: pd.DataFrame, column: str, n: int,ref_id):
    df = df.copy()
    for row in df.iterrows():
        if row["ID"] == ref_id:
            row["overlap"] = 0
        else :
            row["overlap"] = overlap_procentage(df[df["ID"] == ref_id],row[column])

    return df.sort_values(by='overlap', ascending=False).head(n)

IndentationError: expected an indented block (1866525710.py, line 14)

In [ ]:
df_overlap = get_best_overlap(df_bow,"bow",10,66)
df_overlap